# EE 559 Homework 6

## Name: Ishaan Vasant 
## USC ID: 6989-5065-37

### 1. Multi-class and Multi-Label Classification Using Support Vector Machines

### (a) Download the Anuran Calls (MFCCs) Data Set from: https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs). Choose 70% of the data randomly as the training set.


In [83]:
import pandas as pd
import numpy as np

In [84]:
cd /Users/ishaan/Desktop/Anuran Calls (MFCCs)

/Users/ishaan/Desktop/Anuran Calls (MFCCs)


In [85]:
df= pd.read_csv("Frogs_MFCCs.csv")

In [87]:
X= df.drop(['Family','Genus','Species','RecordID'], axis=1)
Y1= df.Family
Y2=df.Genus
Y3=df.Species
frames=[Y1,Y2,Y3]
Y=pd.concat(frames,axis=1)

In [88]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3)

In [89]:
Y1train=Ytrain.Family
Y2train=Ytrain.Genus
Y3train=Ytrain.Species
Y1test=Ytest.Family
Y2test=Ytest.Genus
Y3test=Ytest.Species

### (b) Each instance has three labels: Families, Genus, and Species. Each of the labels has multiple classes. We wish to solve a multi-class and multi-label problem. One of the most important approaches to multi-class classification is to train a classifier for each label. We first try this approach:

### i. Research exact match and hamming score/ loss methods for evaluating multi- label classification and use them in evaluating the classifiers in this problem.

#### Exact match is calculated using accuracy_score() and Hamming loss is calculated using hamming_loss(). These packages do not work for the multilabel problem. Therefore, the following functions were defined:-

In [218]:
def accuracy(M,N):
    count=0
    for i in range(0,len(M)):
        if (M.iloc[i,0]==N.iloc[i,0] and M.iloc[i,1]==N.iloc[i,1] and M.iloc[i,2]==N.iloc[i,2]):
            count=count+1
    acc=count/len(M)
    return(acc)
        
    
    

In [222]:
def hamming(M,N):
    count1=0
    total=0
    for i in range(0,len(M)):
        count=0
        for j in range(0,3):
            if M.iloc[i,j]==N.iloc[i,j]:
                count=count+1
        count1=count/3
        total=total+count1
    score=total/len(M)
    return(1-score)
        

### ii. Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation.

In [213]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss

In [102]:
from sklearn.model_selection import cross_val_score
Best_CVscore=0
Best_C=0
Best_gamma=0
C=[0.001,0.01,0.1,1,10,100,1000,10000,100000]
for i in C:
    for j in [float(m)/100 for m in range(10,201,10)]:
        model=SVC(kernel='rbf',C=i,gamma=1/(2*j*j))
        CV_score = cross_val_score(model, Xtrain, Y1train, cv=10, scoring='accuracy')  
        Mean_score  =  np.mean(CV_score)
        if Mean_score > Best_CVscore:
            Best_CVscore = Mean_score
            Best_C=i
            Best_gamma=1/(2*j*j)
             
print(Best_C)
print(Best_gamma)
    

10
3.1249999999999996


In [103]:
clf = SVC(kernel='rbf',C=Best_C,gamma=Best_gamma)
clf.fit(Xtrain, Y1train)
Y1predict=clf.predict(Xtest)

In [107]:
Best_CVscore2=0
Best_C2=0
Best_gamma2=0
C=[0.001,0.01,0.1,1,10,100,1000,10000,100000]
for i in C:
    for j in [float(m)/100 for m in range(10,201,10)]:
        model=SVC(kernel='rbf',C=i,gamma=1/(2*j*j))
        CV_score = cross_val_score(model, Xtrain, Y2train, cv=10, scoring='accuracy')  
        Mean_score  =  np.mean(CV_score)
        if Mean_score > Best_CVscore2:
            Best_CVscore2 = Mean_score
            Best_C2=i
            Best_gamma2=1/(2*j*j)
             
print(Best_C2)
print(Best_gamma2)

10
3.1249999999999996


In [108]:
clf = SVC(kernel='rbf',C=Best_C2,gamma=Best_gamma2)
clf.fit(Xtrain, Y2train)
Y2predict=clf.predict(Xtest)

In [111]:
Best_CVscore3=0
Best_C3=0
Best_gamma3=0
C=[0.001,0.01,0.1,1,10,100,1000,10000,100000]
for i in C:
    for j in [float(m)/100 for m in range(10,201,10)]:
        model=SVC(kernel='rbf',C=i,gamma=1/(2*j*j))
        CV_score = cross_val_score(model, Xtrain, Y3train, cv=10, scoring='accuracy')  
        Mean_score  =  np.mean(CV_score)
        if Mean_score > Best_CVscore3:
            Best_CVscore3 = Mean_score
            Best_C3=i
            Best_gamma3=1/(2*j*j)
             
print(Best_C3)
print(Best_gamma3)

10
2.0


In [112]:
clf = SVC(kernel='rbf',C=Best_C3,gamma=Best_gamma3)
clf.fit(Xtrain, Y3train)
Y3predict=clf.predict(Xtest)

In [215]:
Y1predict=pd.Series(Y1predict)
Y2predict=pd.Series(Y2predict)
Y3predict=pd.Series(Y3predict)

frames1=[Y1predict, Y2predict, Y3predict]
YpredictA=pd.concat(frames1,axis=1)

In [219]:
accuracy(Ytest,YpredictA)

0.9087540528022232

In [223]:
hamming(Ytest,YpredictA)

0.05635324996140134

#### Exact match = 0.9087540528022232
#### Hamming Loss = 0.05635324996140134

### iii. Repeat 1(b)ii with L1-penalized SVMs.

#### The values in the CSV file have already been normalized.

In [115]:
from sklearn.svm import LinearSVC

In [117]:
Best_CVscore4=0
Best_C4=0

C=[0.001,0.01,0.1,1,10,100,1000,10000,100000]
for i in C:
    model=LinearSVC(penalty='l1',dual=False, C=i)
    CV_score = cross_val_score(model, Xtrain, Y1train, cv=10, scoring='accuracy')  
    Mean_score  =  np.mean(CV_score)
    if Mean_score > Best_CVscore4:
        Best_CVscore4 = Mean_score
        Best_C4=i
             
print(Best_C4)

1


In [129]:
clf = LinearSVC(penalty='l1',dual=False,C=Best_C4)
clf.fit(Xtrain, Y1train)
Y4predict=clf.predict(Xtest)

In [121]:
Best_CVscore5=0
Best_C5=0

C=[0.001,0.01,0.1,1,10,100,1000,10000,100000]
for i in C:
    model=LinearSVC(penalty='l1',dual=False,C=i)
    CV_score = cross_val_score(model, Xtrain, Y2train, cv=10, scoring='accuracy')  
    Mean_score  =  np.mean(CV_score)
    if Mean_score > Best_CVscore5:
        Best_CVscore5 = Mean_score
        Best_C5=i
             
print(Best_C5)

100000


In [132]:
clf = LinearSVC(penalty='l1',dual=False,C=Best_C5)
clf.fit(Xtrain, Y2train)
Y5predict=clf.predict(Xtest)

In [125]:
Best_CVscore6=0
Best_C6=0

C=[0.001,0.01,0.1,1,10,100,1000,10000,100000]
for i in C:
    model=LinearSVC(penalty='l1',dual=False,C=i)
    CV_score = cross_val_score(model, Xtrain, Y3train, cv=10, scoring='accuracy')  
    Mean_score  =  np.mean(CV_score)
    if Mean_score > Best_CVscore6:
        Best_CVscore6 = Mean_score
        Best_C6=i
             
print(Best_C6)

100


In [135]:
clf = LinearSVC(penalty='l1',dual=False,C=Best_C6)
clf.fit(Xtrain, Y3train)
Y6predict=clf.predict(Xtest)

In [216]:
Y4predict=pd.Series(Y4predict)
Y5predict=pd.Series(Y5predict)
Y6predict=pd.Series(Y6predict)

frames2=[Y4predict, Y5predict, Y6predict]
YpredictB=pd.concat(frames2,axis=1)

In [220]:
accuracy(Ytest,YpredictB)

0.9092172301991662

In [224]:
hamming(Ytest,YpredictB)

0.05604446503010596

#### Exact Match = 0.9092172301991662
#### Hamming Loss = 0.05604446503010596

### iv. Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. Report your conclusions about the classifiers you trained.

#### SMOTE technique was used.

In [141]:
from imblearn.over_sampling import SMOTE 

In [144]:
sm=SMOTE()
X_res1, Y1_res = sm.fit_sample(Xtrain, Y1train)
X_res2, Y2_res = sm.fit_sample(Xtrain, Y2train)
X_res3, Y3_res = sm.fit_sample(Xtrain, Y3train)



In [145]:
Best_CVscore7=0
Best_C7=0

C=[0.001,0.01,0.1,1,10,100,1000,10000,100000]
for i in C:
    model=LinearSVC(penalty='l1',dual=False,C=i)
    CV_score = cross_val_score(model, X_res1, Y1_res, cv=10, scoring='accuracy')  
    Mean_score  =  np.mean(CV_score)
    if Mean_score > Best_CVscore7:
        Best_CVscore7 = Mean_score
        Best_C7=i
             
print(Best_C7)

clf = LinearSVC(penalty='l1',dual=False,C=Best_C7)
clf.fit(X_res1, Y1_res)
Y7predict=clf.predict(Xtest)

100000


In [148]:
Best_CVscore8=0
Best_C8=0

C=[0.001,0.01,0.1,1,10,100,1000,10000,100000]
for i in C:
    model=LinearSVC(penalty='l1',dual=False,C=i)
    CV_score = cross_val_score(model, X_res2, Y2_res, cv=10, scoring='accuracy')  
    Mean_score  =  np.mean(CV_score)
    if Mean_score > Best_CVscore8:
        Best_CVscore8 = Mean_score
        Best_C8=i
             
print(Best_C8)

clf = LinearSVC(penalty='l1',dual=False,C=Best_C8)
clf.fit(X_res2, Y2_res)
Y8predict=clf.predict(Xtest)

1000


In [151]:
Best_CVscore9=0
Best_C9=0

C=[0.001,0.01,0.1,1,10,100,1000,10000,100000]
for i in C:
    model=LinearSVC(penalty='l1',dual=False,C=i)
    CV_score = cross_val_score(model, X_res3, Y3_res, cv=10, scoring='accuracy')  
    Mean_score  =  np.mean(CV_score)
    if Mean_score > Best_CVscore9:
        Best_CVscore9 = Mean_score
        Best_C9=i
             
print(Best_C9)

clf = LinearSVC(penalty='l1',dual=False,C=Best_C9)
clf.fit(X_res3, Y3_res)
Y9predict=clf.predict(Xtest)

100


In [217]:
Y7predict=pd.Series(Y7predict)
Y8predict=pd.Series(Y8predict)
Y9predict=pd.Series(Y9predict)

frames3=[Y7predict, Y8predict, Y9predict]
YpredictC=pd.concat(frames3,axis=1)

In [221]:
accuracy(Ytest,YpredictC)

0.8550254747568319

In [225]:
hamming(Ytest,YpredictC)

0.07379959857958795

#### Exact Match = 0.8550254747568319
#### Hamming Loss = 0.07379959857958795

#### Conclusions:-

#### a) For SVM using Gaussian kernels and one versus all classifiers:-

#### Exact match = 0.9087540528022232
#### Hamming Loss = 0.05635324996140134

#### b) For L1-penalized SVM:-

#### Exact Match = 0.9092172301991662
#### Hamming Loss = 0.05604446503010596

#### c) For L1-penalized SVM using SMOTE technique:-

#### Exact Match = 0.8550254747568319
#### Hamming Loss = 0.07379959857958795

#### Based on these observations, the exact match and hamming loss is similar for SVM using Gaussian kernels and L1-penalized SVM. When SMOTE technique is used, The exact match comes down drastically and hamming loss increases.